# Estimate noise model parameters

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import thztools as thz

In [ ]:
rng = np.random.default_rng(0)
n, m, dt = 256, 50, 0.05
t = thz.timebase(n, dt=dt)
mu = thz.wave(n, dt=dt)
alpha, beta, tau = 1e-4, 1e-2, 1e-3
noise_model = thz.NoiseModel(sigma_alpha=alpha, sigma_beta=beta, sigma_tau=tau, dt=dt)
a = 1.0 + 1e-2 * np.concatenate(([0.0], rng.standard_normal(m - 1)))
eta = 1e-3 * np.concatenate(([0.0], rng.standard_normal(m - 1)))
# Orient the array columnwise
z = thz.scaleshift(np.repeat(np.atleast_2d(mu), m, axis=0), dt=dt, a=a, eta=eta).T
x = z + noise_model.noise_sim(z, axis=0, seed=12345)
noise_res = thz.noisefit(x, sigma_alpha0=alpha, sigma_beta0=beta, sigma_tau0=tau, dt=dt)
noise_res.noise_model